In [9]:
%%capture
! pip install "datasets" "pytorch-lightning" "wandb" "torcheval" "torchmetrics" "clifford" "torch-geometric"

In [10]:
import torch
import numpy as np
import torch.nn as nn
from torch.nn import functional as F

#import torch_geometric.nn

import pytorch_lightning as pl
from pytorch_lightning import LightningDataModule, LightningModule, Trainer, seed_everything

from torchmetrics.classification import BinaryAccuracy
from torchmetrics.classification import BinaryPrecision
from torchmetrics.classification import BinaryRecall
from torchmetrics.classification import BinaryF1Score

from clifford import Cl


In [11]:
use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")

# GATr ARCHITECTURE

---
## Claudio's section
---

---
## Jacopo's section
---

### Auxiliar / dummy

In [12]:
dummy_mv = {'': 1.0, 'e1': 1.01, 'e2': 1.02, 'e3': 1.03, 'e4': 1.04, 'e12': 2.01, 'e13': 2.02, 'e14': 2.03, 'e23': 2.04, 'e24': 2.05, 'e34': 2.06,  'e123': 3.01, 'e124': 3.02, 'e134': 3.03,  'e234': 3.04, 'e1234': 5.0}

In [13]:
dummy_tensor = torch.tensor([0.1, 0.2, 0.3, 0.4, 0.5], requires_grad=True)

In [14]:
dummy_matrix = torch.tensor([[-0.0281, -0.0399,  0.0262, -0.0089],
        [-0.0399, -0.0567,  0.0373, -0.0127],
        [ 0.0262,  0.0373, -0.0245,  0.0083],
        [-0.0089, -0.0127,  0.0083, -0.0028]], requires_grad=True)

In [15]:
def compose_multivector(sample):
    wss_multivector = 0
    for row in sin_sample['wss'][()]:
        vector = row[0] * blades[''] + row[1] * blades[''] + row[2] * blades['']
        wss_multivector += vector

    # pressure as a pseudoscalar
    pressure_multivector = 0
    for elem in sin_sample['pressure'][()]:
        vector = elem * blades['e1234']
        pressure_multivector += vector

      # pos as a point
    pos_multivector = 0
    for row in sin_sample['pos'][()]:
        vector = row[0] * blades['e123'] + row[1] * blades['e124'] + row[2] * blades['e134']
        pos_multivector += vector

      # face as a plane
    face_multivector = 0
    for row in sin_sample['face'][()]:
        vector = row[0] * blades['e1'] + row[1] * blades['e2'] + row[2] * blades['e3']
        face_multivector += vector

      # inlet as a scalar
    inlet_multivector = 0
    for elem in sin_sample['inlet_idcs'][()]:
        vector = elem * blades['']
        inlet_multivector += vector


    total_multivector = wss_multivector + pressure_multivector + pos_multivector + face_multivector + inlet_multivector
    return total_multivector

### Layers

In [16]:
class J_equi_linear(pl.LightningModule):
    def __init__(self, mv_dict_type, in_feat=5, out_feat=5, ):
        super(J_equi_linear, self).__init__()

        self.mv_dict_type = mv_dict_type # indicate if the multivector is expressed as 16 basis or they are all summed up
        self.equi_linear_layer = torch.nn.Linear(in_features=in_feat, out_features=out_feat)

    def forward(self, multivector):

        if self.mv_dict_type == True: # first equivariant layer of the architecture
            multivector_basis = list( multivector.keys() )

            # summing up all scalars - vectorl - bivectorl - trivectorl - bias ( its '' basis )
            bias = 0                     # should be: 1.0
            vector_value = 0             # should be: 4.1
            bivector_value = 0           # should be: 12.21
            trivector_value = 0          # should be: 12.10
            pseudoscalar_value = 0       # should be: 5.0

            for basis in multivector_basis:
                if len(basis) == 2:     vector_value += multivector[basis]
                elif len(basis) == 3:   bivector_value += multivector[basis]
                elif len(basis) == 4:   trivector_value += multivector[basis]
                elif len(basis) == 5:   pseudoscalar_value += multivector[basis]
                else:                   bias += multivector[basis]

            # check correctness of position of each entries. Maybe switch bias and scalar value (?)
            input_tensor = torch.tensor( [ bias, vector_value, bivector_value, trivector_value, pseudoscalar_value ] ,
                                            dtype=torch.float32,
                                            requires_grad=True
                                        )

            # check why print 12.2099999 instead of 12.1, can be a problem ?
            #print(f"--Analizing input component:\nbias(''): {bias}\nvector(e_i): {vector_value}\nbivector(e_ij): {bivector_value}\ntrivector(e_ijk): {trivector_value}\npseudoscalar(e_ijkl): {pseudoscalar_value}")
            #print(f"\ninput_tensor: {input_tensor}\n--input's analisys finished. \n\n")
        else:
            input_tensor = multivector

        output = self.equi_linear_layer( input_tensor )

        return output


my_layer1 = J_equi_linear(mv_dict_type=True)
out = my_layer1(dummy_mv)
print(out)

my_layer2 = J_equi_linear(mv_dict_type=False)
out2 = my_layer2(dummy_tensor)
print(out2)

tensor([ 0.5496, -5.0536,  0.8110, -3.9570,  1.7721], grad_fn=<ViewBackward0>)
tensor([ 0.2753, -0.2292,  0.1056,  0.2185,  0.5455], grad_fn=<ViewBackward0>)


In [17]:
class J_norm_linear(pl.LightningModule):
    def __init__(self, normalized_shape = (5,), eps=1e-05):
        super(J_norm_linear, self).__init__()

        self.normalized_shape = normalized_shape # shape of the tesnor
        self.eps = eps

    def forward(self, multivector):
        output = torch.nn.functional.layer_norm( multivector, self.normalized_shape, eps = self.eps )

        return output

my_layer = J_norm_linear()
out = my_layer(dummy_tensor)
#print(out)

In [18]:
class J_geo_attention(pl.LightningModule):
    def __init__(self):
        super(J_geo_attention, self).__init__()
        #https://pytorch.org/docs/stable/generated/torch.nn.functional.scaled_dot_product_attention.html
        #https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html

        tensor_length = 5
        self.key   = nn.Parameter(torch.randn(tensor_length)).unsqueeze(0) # it is correct do it here? it is request to use 2 dimension
        self.value = nn.Parameter(torch.randn(tensor_length)).unsqueeze(0)

    def forward(self, multivector_query):
        multivector_query = multivector_query.unsqueeze(0) # it is request to use 2 dimension
        output = torch.nn.functional.scaled_dot_product_attention(multivector_query, self.key, self.value)

        return output

my_layer = J_geo_attention()
out = my_layer(dummy_tensor)
print(out)

tensor([[ 1.0811,  0.7897, -0.0917,  0.4805, -1.5568]], grad_fn=<MmBackward0>)


In [19]:
# check the correctness!

class J_gated_relu(pl.LightningModule):
    def __init__(self):
        super(J_gated_relu, self).__init__()

        # gelu: https://paperswithcode.com/method/gelu
        # gated_gelu(x,g) = g*GELU(x) + (1-g)*x  ( from chatgpt )

        self.gelu_fn = nn.GELU()

    def forward(self, multivector):

        scalar = torch.tensor(multivector[:, 0]) # taking scalar component
        #cut_multivector = torch.tensor(multivector[:,1:]) # exlude first elemnt ( scalar )

        gelu_value = self.gelu_fn(scalar)
        output = gelu_value * multivector

        return output

my_layer = J_gated_relu()
out = my_layer(dummy_matrix)
#print(out)

<ipython-input-19-a904f204d5d5>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  scalar = torch.tensor(multivector[:, 0]) # taking scalar component


In [ ]:
class J_geo_bilin(pl.LightningModule):
    def __init__(self):
        super(J_geo_bilin, self).__init__()

    def forward(self, multivector):
        scalar = multivector[-1] # taking pseudoscalar component
        #cut_multivector = multivector[:-1] # exlude last element

        transposed_multivector = multivector.t()
        inner_product = torch.matmul( multivector, transposed_multivector )
        outer_product = torch.ger( transposed_multivector, multivector )

        output = scalar * outer_product
        return output

my_layer = J_geo_bilin()
out = my_layer(dummy_tensor)
print(out)

In [26]:
class J_gatr(pl.LightningModule):
    def __init__(self):
        super().__init__()

        #self.device = must run on cpu and gpu, create first block with device cell

        # input layer: in = input
        self.in_equi_linear_layer = J_equi_linear(in_feat=5, out_feat=5, mv_dict_type=True )

        # first half block: h1 = first half
        self.h1_norm_linear_layer   = J_norm_linear(normalized_shape=(5,))
        self.h1_equi_linear_layer_1 = J_equi_linear(in_feat=5, out_feat=5, mv_dict_type=False )
        self.h1_att_layer           = J_geo_attention()
        self.h1_equi_linear_layer_2 = J_equi_linear(in_feat=5, out_feat=5, mv_dict_type=False )

        # second half block: h2 = second half
        self.h2_norm_linear_layer   = J_norm_linear(normalized_shape=(5,))
        self.h2_equi_linear_layer_1 = J_equi_linear(in_feat=5, out_feat=5, mv_dict_type=False )
        self.h2_geo_bilinear_layer  = J_geo_bilin()
        self.h2_gated_relu_layer    = J_gated_relu()
        self.h2_equi_linear_layer_2 = J_equi_linear(in_feat=5, out_feat=5, mv_dict_type=False )


        # output layer out = output
        self.out_equi_linear_layer = J_equi_linear(in_feat=5, out_feat=5, mv_dict_type=False)

        # metrics
        self.accuracy_metric  = BinaryAccuracy()
        self.precision_metric = BinaryPrecision()
        self.recall_metric    = BinaryRecall()
        self.f1score_metric   = BinaryF1Score()


    def forward(self, multivector):
        do_print = True

        print(f"input: {multivector.values()}\n\nFirst block")
        # input layer
        in_out_eq  = self.in_equi_linear_layer(multivector);       print(f"in_out_eq: {in_out_eq}")

        # first half block
        print(f"\n\nFirstblock")
        h1_out_norm = self.h1_norm_linear_layer( in_out_eq );        print(f"h1_out_norm: {h1_out_norm}")
        h1_out_eq_1 = self.h1_equi_linear_layer_1( h1_out_norm );    print(f"h1_out_eq_1: {h1_out_eq_1}")
        h1_out_att  = self.h1_att_layer( h1_out_eq_1 ).squeeze();    print(f"h1_out_att : {h1_out_att}") # I output I have [[...]], I think is better have [...]
        h1_out_eq_2 = self.h1_equi_linear_layer_2( h1_out_att );     print(f"h1_out_eq_2: {h1_out_eq_2}")

        # second half block
        print(f"\n\nSecond block")
        h2_out_norm       = self.h2_norm_linear_layer( h1_out_eq_2 + in_out_eq );   print(f"h2_out_norm:       {h2_out_norm}")
        h2_out_eq_1       = self.h2_equi_linear_layer_1( h2_out_norm );             print(f"h2_out_eq_1:       {h2_out_eq_1}")
        h2_out_geo_bilin  = self.h2_geo_bilinear_layer( h2_out_eq_1 );              print(f"h2_out_geo_bilin:  {h2_out_geo_bilin}")
        #h2_out_geo_bilin  = h2_out_eq_1 # dummy
        h2_out_gated_relu = self.h2_gated_relu_layer( h2_out_geo_bilin );           print(f"h2_out_gated_relu: {h2_out_gated_relu}")
        h2_out_eq_2       = self.h2_equi_linear_layer_2( h2_out_gated_relu );       print(f"h2_out_eq_2:       {h2_out_eq_2}")


        # output of the whole block
        output_block = h2_out_eq_2 + ( in_out_eq + h1_out_eq_2 ); print(f"output_block: {output_block}")
        #here we could 2 things:
        # - added the scalar component at gated_relu to pass from shape 4 to 5 ( which I have done)
        # - to this addition adding withouth tensor, so summing up with list the first 4 component and last one


        # output layer
        print("\n\nOutput block")
        output = self.out_equi_linear_layer(output_block);   print(f"gatr_output: {output}")

        return output

    def training_step(self, batch):
        x, y = batch

        # check if the used loss is correct according to the paper, find it on the github
        loss = F.binary_cross_entropy(x, y) # In case we need to adjust dimension
        #loss = F.binary_cross_entropy(self(x).view(-1), y.float())

        #predictions = self.forward(x).long().squeeze()
        #y = torch.tensor(y, dtype=torch.long)

        #accuracy  = accuracy(predictions, y)
        #precision = self.precision_metric(predictions, y)
        #recall    = self.recall_metric(predictions, y)
        #f1_score  = self.f1_metric(predictions, y)

        #wandb.log({"acc": accuracy,"loss": loss,"precision": precision, "recall": recall, "f1-score:":f1_score })

        return loss

    '''
    # look at https://colab.research.google.com/drive/1lUIrtEiQN9hA5RIdKTpUS7w5gI1DLvOl#scrollTo=gigZq4h0yifA
    def validation_step(self, batch, batch_idx):
        return None

    def test_step(self, batch, batch_idx):
        return None
    '''
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

gatr_j = J_gatr()

In [27]:
out = gatr_j(dummy_mv)
print(f"\noutput: {out}")

input: dict_values([1.0, 1.01, 1.02, 1.03, 1.04, 2.01, 2.02, 2.03, 2.04, 2.05, 2.06, 3.01, 3.02, 3.03, 3.04, 5.0])

First block
in_out_eq: tensor([-1.2836, -0.1197, -4.2580, -8.2513, -5.5661], grad_fn=<ViewBackward0>)


Firstblock
h1_out_norm: tensor([ 0.8910,  1.2880, -0.1236, -1.4857, -0.5698],
       grad_fn=<NativeLayerNormBackward0>)
h1_out_eq_1: tensor([-1.1693,  0.8410, -0.2879,  0.6833, -1.0619], grad_fn=<ViewBackward0>)
h1_out_att : tensor([ 0.3529, -1.8721,  0.2947, -0.8392,  0.5916],
       grad_fn=<SqueezeBackward0>)
h1_out_eq_2: tensor([-0.2722,  0.2284, -0.7427,  0.0522, -0.1119], grad_fn=<ViewBackward0>)


Second block
h2_out_norm:       tensor([ 0.8431,  1.4024, -0.3144, -1.3891, -0.5420],
       grad_fn=<NativeLayerNormBackward0>)
h2_out_eq_1:       tensor([ 0.0808, -0.4473, -0.6555,  0.1801, -0.4252], grad_fn=<ViewBackward0>)
h2_out_geo_bilin:  tensor([[-0.0028,  0.0154,  0.0225, -0.0062,  0.0146],
        [ 0.0154, -0.0851, -0.1247,  0.0343, -0.0809],
        [ 0.022

<ipython-input-19-a904f204d5d5>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  scalar = torch.tensor(multivector[:, 0]) # taking scalar component


---
## Lorenzo's section
---